In [1]:
from __future__ import print_function

from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import exists

import datetime
import numpy as np
import glob

from multiprocessing import Pool

## Boilerplate path hack to give access to full clustered_SNe package
import os,sys
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, parse_run, \
                                         Inputs
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs



/Users/egentry/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


#### Helpful resource:
http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [2]:
class Momentum_Surface_Point(object):
    def __init__(self, simulation):
        self.mass = simulation.cluster_mass
        self.momentum = simulation.momentum
        self.density = simulation.background_density
        self.metallicity = simulation.metallicity
        self.id = simulation.id

In [3]:
def parse_into_simulations(data_dir_and_id):
    # wrapper function to allow multiprocessing + map
    data_dir, id = data_dir_and_id
    last_run = parse_run(data_dir, id)
    simulation = Simulation.from_last_run(data_dir, last_run)
    
    inputs = Inputs(os.path.join(data_dir, id + "_inputs.dat"))
    simulation_inputs = Simulation_Inputs.from_Inputs(id, inputs)
    return simulation, simulation_inputs
        

In [4]:
for sim in session.query(Simulation):
    print(sim)

In [5]:
for sim_inputs in session.query(Simulation_Inputs):
    print(sim_inputs)

In [7]:
def add_entire_dir_to_database(data_dir):
    overview_filenames = glob.glob(os.path.join(data_dir, "*overview.dat"))
    ids = [None]*len(overview_filenames)
    for k, filename in enumerate(overview_filenames):
        ids[k] = os.path.basename(filename).split("_overview.dat")[0]
#     print(ids)

    mappable_inputs = [(data_dir, id) for id in ids][0:4]
    
    pool = Pool(4)
    simulation_tuples = pool.map(parse_into_simulations, mappable_inputs)
    for simulation, simulation_inputs in simulation_tuples:
        simulation.add_or_update_to_table()
        simulation_inputs.add_or_update_to_table()
add_entire_dir_to_database("../saved_runs/cluster_parameter_study_new")

In [ ]:
session.commit()

In [3]:
session.close()

# Use the database

In [8]:
points = []

for simulation in session.query(Simulation):
    point = Momentum_Surface_Point(simulation)
    points.append(point)

In [9]:
momenta = [point.momentum for point in points if (point.metallicity==metallicity_solar)]
masses = [point.mass for point in points if (point.metallicity==metallicity_solar)]
densities = [point.density for point in points if (point.metallicity==metallicity_solar)]

momenta = np.array(momenta)
masses = np.array(masses)
densities = np.array(densities)

momenta = momenta / masses / 1e5
masses = masses / M_solar
densities

masses_set = set()
for mass in masses:
    masses_set.add(mass)

density_set = set()
for density in densities:
    density_set.add(density)
    
masses_1D = np.sort(list(masses_set))
densities_1D = np.sort(list(density_set))

masses_2D, densities_2D = np.meshgrid(masses_1D, densities_1D) 

momenta_2D = np.empty_like(masses_2D)

for i,mass in enumerate(masses_1D):
    for j,density in enumerate(densities_1D):
        good_masses = (masses == mass)
        good_densities = (densities == density)
        good = good_masses & good_densities
        print("good momenta: ", momenta[good])
        print("mass: {0}   density:  {1}".format(mass, density))
        print()
        momenta_2D[j,i] = np.average(momenta[good])
    
momenta_2D[momenta_2D == 0] = 1

In [12]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


surf = ax.plot_wireframe(np.log10(masses_2D), np.log10(densities_2D), momenta_2D, rstride=1, cstride=1,
                       linewidth=1)

plt.xlabel("Mass")
plt.ylabel("density")
plt.show()